In [1]:
import nltk
from nltk import tokenize
from nltk.tokenize import TweetTokenizer
from stop_words import get_stop_words
from nltk.stem.porter import PorterStemmer
from gensim import corpora, models
import pandas as pd
import nltk
import numbers
from nltk.sentiment.vader import SentimentIntensityAnalyzer

tknzr = TweetTokenizer()


/Users/elenajia/anaconda/lib/python3.6/site-packages/nltk/twitter/__init__.py:20: UserWarning: The twython library has not been installed. Some functionality from the twitter package will not be available.
  warnings.warn("The twython library has not been installed. "


In [2]:
# import sys
# spacy.en.download

In [3]:
# spacy
import spacy

nlp = spacy.load('en')
doc = nlp('They told us to duck.')

from spacy.en import English
parser = English()

### test spacy

### test

In [4]:

multiSentence = "There is an art, it says, or rather, a knack to flying." \
                 "The knack lies in learning how to throw yourself at the ground and miss." \
                 "In the beginning the Universe was created. This has made a lot of people "\
                 "very angry and been widely regarded as a bad move."

# all you have to do to parse text is this:
#note: the first time you run spaCy in a file it takes a little while to load up its modules
parsedData = parser(multiSentence)

# Let's look at the tokens
# All you have to do is iterate through the parsedData
# Each token is an object with lots of different properties
# A property with an underscore at the end returns the string representation
# while a property without the underscore returns an index (int) into spaCy's vocabulary
# The probability estimate is based on counts from a 3 billion word
# corpus, smoothed using the Simple Good-Turing method.
for i, token in enumerate(parsedData):
    print("original:", token.orth, token.orth_)
    print("lowercased:", token.lower, token.lower_)
    print("lemma:", token.lemma, token.lemma_)
    print("shape:", token.shape, token.shape_)
    print("prefix:", token.prefix, token.prefix_)
    print("suffix:", token.suffix, token.suffix_)
    print("log probability:", token.prob)
    print("Brown cluster id:", token.cluster)
    print("----------------------------------------")
    if i > 1:
        break
# Let's look at the sentences
sents = []
# the "sents" property returns spans
# spans have indices into the original string
# where each index value represents a token
for span in parsedData.sents:
    # go from the start to the end of each span, returning each token in the sentence
    # combine each token using join()
    sent = ''.join(parsedData[i].string for i in range(span.start, span.end)).strip()
    sents.append(sent)

for sentence in sents:
    print(sentence)

original: 640 There
lowercased: 530 there
lemma: 530 there
shape: 489815 Xxxxx
prefix: 2907 T
suffix: 48458 ere
log probability: -7.347356796264648
Brown cluster id: 1918
----------------------------------------
original: 474 is
lowercased: 474 is
lemma: 488 be
shape: 21581 xx
prefix: 570 i
suffix: 474 is
log probability: -4.457748889923096
Brown cluster id: 762
----------------------------------------
original: 523 an
lowercased: 523 an
lemma: 523 an
shape: 21581 xx
prefix: 469 a
suffix: 523 an
log probability: -6.014852046966553
Brown cluster id: 3
----------------------------------------
There is an art, it says, or rather, a knack to flying.
The knack lies in learning how to throw yourself at the ground and miss.
In the beginning the Universe was created.
This has made a lot of people very angry and been widely regarded as a bad move.


In [5]:
# Let's look at the part of speech tags of the first sentence
for span in parsedData.sents:
    sent = [parsedData[i] for i in range(span.start, span.end)]
    break

for token in sent:
    print(token.orth_, token.pos_)

There ADV
is VERB
an DET
art NOUN
, PUNCT
it PRON
says VERB
, PUNCT
or CONJ
rather ADV
, PUNCT
a DET
knack NOUN
to ADP
flying NOUN
. PUNCT


In [6]:
# Let's look at the dependencies of this example:
example = "The boy with the spotted dog quickly ran after the firetruck."
parsedEx = parser(example)
# shown as: original token, dependency tag, head word, left dependents, right dependents
for token in parsedEx:
    print(token.orth_, token.dep_, token.head.orth_, [t.orth_ for t in token.lefts], [t.orth_ for t in token.rights])

The det boy [] []
boy nsubj ran ['The'] ['with']
with prep boy [] ['dog']
the det dog [] []
spotted amod dog [] []
dog pobj with ['the', 'spotted'] []
quickly advmod ran [] []
ran ROOT ran ['boy', 'quickly'] ['after', '.']
after prep ran [] ['firetruck']
the det firetruck [] []
firetruck pobj after ['the'] []
. punct ran [] []


In [7]:
### random
# https://github.com/nltk/nltk_book/blob/master/book/ch08.rst

In [8]:
# Let's look at the named entities of this example:
example = "Apple's stocks dropped dramatically after the death of Steve Jobs in October."
parsedEx = parser(example)
for token in parsedEx:
    print(token.orth_, token.ent_type_ if token.ent_type_ != "" else "(not an entity)")

print("-------------- entities only ---------------")
# if you just want the entities and nothing else, you can do access the parsed examples "ents" property like this:
ents = list(parsedEx.ents)
for entity in ents:
    print(entity.label, entity.label_, ' '.join(t.orth_ for t in entity))

Apple ORG
's (not an entity)
stocks (not an entity)
dropped (not an entity)
dramatically (not an entity)
after (not an entity)
the (not an entity)
death (not an entity)
of (not an entity)
Steve PERSON
Jobs PERSON
in (not an entity)
October DATE
. (not an entity)
-------------- entities only ---------------
349 ORG Apple
346 PERSON Steve Jobs
356 DATE October


In [9]:
test_n = "medicine med meds Elena Jobs Summer"
parsed_test_n = parser(test_n)
for token in parsed_test_n:
    print(token.orth_, token.ent_type_ if token.ent_type_ != "" else "(not an entity)")

medicine (not an entity)
med (not an entity)
meds (not an entity)
Elena PERSON
Jobs PERSON
Summer PERSON


In [10]:
messyData = "lol that is rly funny :) This is gr8 i rate it 8/8!!!"
parsedData = parser(messyData)
for token in parsedData:
    print(token.orth_, token.pos_, token.lemma_)
    
# it does pretty well! Note that it does fail on the token "gr8", 
# taking it as a verb rather than an adjective meaning "great"
# and "lol" probably isn't a noun...it's more like an interjection

lol NOUN lol
that ADJ that
is VERB be
rly ADV rly
funny ADJ funny
:) PUNCT :)
This DET this
is VERB be
gr8 VERB gr8
i PRON i
rate VERB rate
it PRON it
8/8 NUM 8/8
! PUNCT !
! PUNCT !
! PUNCT !


In [11]:
## another tutorial
## http://blog.sharepointexperience.com/2016/01/nlp-and-sharepoint-part-1/

In [12]:
review = '''The Road-550-W from Adventure Works Cycles is everything it's advertised to be. Finally, a quality bike that is actually built for a woman and provides control and comfort in one neat package. The top tube is shorter, the suspension is weight-tuned and there's a much shorter reach to the brake levers. All this adds up to a great mountain bike that is sure to accommodate any woman's anatomy. In addition to getting the size right, the saddle is incredibly comfortable. Attention to detail is apparent in every aspect from the frame finish to the careful design of each component. Each component is a solid performer without any fluff. The designers clearly did their homework and thought about size, weight, and funtionality throughout. And at less than 19 pounds, the bike is manageable for even the most petite cyclist. We had 5 riders, including my good friend Dr. Joseph A. Bicycle, and his wife, Mrs. Jane Bicycle, take the bike out for a spin and really put it to the test. The results were consistent and very positive. Our testers loved the manuverability and control they had with the redesigned frame on the 550-W. A definite improvement over the 2002 design. Four out of five testers listed quick handling and responsivness were the key elements they noticed. Technical climbing and on the flats, the bike just cruises through the rough. Tight corners and obstacles were handled effortlessly. The fifth tester was more impressed with the smooth ride. The heavy-duty shocks absorbed even the worst bumps and provided a soft ride on all but the nastiest trails and biggest drops. The shifting was rated superb and typical of what we've come to expect from Adventure Works Cycles. On descents, the bike handled flawlessly and tracked very well. The bike is well balanced front-to-rear and frame flex was minimal. In particular, the testers noted that the brake system had a unique combination of power and modulation.  While some brake setups can be overly touchy, these brakes had a good amount of power, but also a good feel that allows you to apply as little or as much braking power as is needed. Second is their short break-in period. We found that they tend to break-in well before the end of the first ride; while others take two to three rides (or more) to come to full power. On the negative side, the pedals were not quite up to our tester's standards. Just for fun, we experimented with routine maintenance tasks. Overall we found most operations to be straight forward and easy to complete. The only exception was replacing the front wheel. The maintenance manual that comes with the bike say to install the front wheel with the axle quick release or bolt, then compress the fork a few times before fastening and tightening the two quick-release mechanisms on the bottom of the dropouts. This is to seat the axle in the dropouts, and if you do not do this, the axle will become seated after you tightened the two bottom quick releases, which will then become loose. It's better to test the tightness carefully or you may notice that the two bottom quick releases have come loose enough to fall completely open. And that's something you don't want to experience while out on the road! The Road-550-W frame is available in a variety of sizes and colors and has the same durable, high-quality aluminum that AWC is known for. At a MSRP of just under $1125.00, it's comparable in price to its closest competitors and we think that after a test drive you'l find the quality and performance above and beyond . You'll have a grin on your face and be itching to get out on the road for more. While designed for serious road racing, the Road-550-W would be an excellent choice for just about any terrain and any level of experience. It's a huge step in the right direction for female cyclists and well worth your consideration and hard-earned money.'''

In [13]:
doc = parser(review)

In [14]:
sentences = [sentence.orth_ for sentence in doc.sents]
print("There were {} sentences found. Here's a sample:".format(len(sentences)))
pd.DataFrame(sentences[8:13])

There were 38 sentences found. Here's a sample:


,0
0,"And at less than 19 pounds, the bike is manage..."
1,"We had 5 riders, including my good friend Dr. ..."
2,The results were consistent and very positive.
3,Our testers loved the manuverability and contr...
4,A definite improvement over the 2002 design.


In [15]:
nounphrases = [[np.orth_, np.root.head.orth_] for np in doc.noun_chunks]
print("There were {} noun phrases found. Here's a sample:".format(len(nounphrases)))
pd.DataFrame(nounphrases[8:18])

There were 156 noun phrases found. Here's a sample:


,0,1
0,one neat package,in
1,The top tube,is
2,the suspension,is
3,a much shorter reach,'s
4,the brake levers,to
5,a great mountain bike,to
6,any woman's anatomy,accommodate
7,addition,In
8,the size,getting
9,the saddle,is


In [16]:
entities = list(doc.ents)
print("There were {} entities found".format(len(entities)))
orgs_and_people = [entity.orth_ for entity in entities if entity.label_ in ['ORG','PERSON']]
pd.DataFrame(orgs_and_people)

There were 19 entities found


,0
0,Adventure Works Cycles
1,Joseph A. Bicycle
2,Jane Bicycle
3,550-W.
4,Adventure Works Cycles
5,AWC


In [17]:
# Write a function that walks up the syntactic tree of the given token and collects all tokens to the root token (including root token).

def tokens_to_root(token):
    """
    Walk up the syntactic tree, collecting tokens to the root of the given `token`.
    :param token: Spacy token
    :return: list of Spacy tokens
    """
    tokens_to_r = []
    while token.head is not token:
        tokens_to_r.append(token)
        token = token.head
        tokens_to_r.append(token)

    return tokens_to_r

# For every token in document, print it's tokens to the root
for token in doc:
    print('{} --> {}'.format(token, tokens_to_root(token)))

# Print dependency labels of the tokens
for token in doc:
    print('-> '.join(['{}-{}'.format(dependent_token, dependent_token.dep_) for dependent_token in tokens_to_root(token)]))


The --> [The, Road-550-W, Road-550-W, is]
Road-550-W --> [Road-550-W, is]
from --> [from, Road-550-W, Road-550-W, is]
Adventure --> [Adventure, Cycles, Cycles, from, from, Road-550-W, Road-550-W, is]
Works --> [Works, Cycles, Cycles, from, from, Road-550-W, Road-550-W, is]
Cycles --> [Cycles, from, from, Road-550-W, Road-550-W, is]
is --> []
everything --> [everything, is]
it --> [it, advertised, advertised, everything, everything, is]
's --> ['s, advertised, advertised, everything, everything, is]
advertised --> [advertised, everything, everything, is]
to --> [to, be, be, advertised, advertised, everything, everything, is]
be --> [be, advertised, advertised, everything, everything, is]
. --> [., is]
Finally --> [Finally, provides]
, --> [,, provides]
a --> [a, bike, bike, provides]
quality --> [quality, bike, bike, provides]
bike --> [bike, provides]
that --> [that, built, built, bike, bike, provides]
is --> [is, built, built, bike, bike, provides]
actually --> [actually, built, built

In [18]:
doc = parser(u'They told us to duck the med.')
for word in doc:
    print(word.text, word.lemma, word.lemma_, word.tag, word.tag_, word.pos, word.pos_)

They 500 they 445 PRP 92 PRON
told 750 tell 455 VBD 97 VERB
us 694 us 445 PRP 92 PRON
to 468 to 452 TO 91 PART
duck 5792 duck 454 VB 97 VERB
the 466 the 426 DT 87 DET
med 4236 med 440 NN 89 NOUN
. 419 . 419 . 94 PUNCT


In [19]:
## https://spacy.io/docs/usage/dependency-parse

# Visualizer
## https://demos.explosion.ai/displacy/

In [20]:
doc = parser(u'I like green eggs and ham.')
for np in doc.noun_chunks:
    print(np.text, np.root.text, np.root.dep_, np.root.head.text)
    # I I nsubj like
    # green eggs eggs dobj like
    # ham ham conj eggs

I I nsubj like
green eggs eggs dobj like
ham ham conj eggs


In [21]:
apples = parser(u'bright red apples on the tree')[2]
print([w.text for w in apples.lefts])
# ['bright', 'red']
print([w.text for w in apples.rights])
# ['on']
assert apples.n_lefts == 2
assert apples.n_rights == 1

['bright', 'red']
['on']


In [22]:
doc = parser(u'I like green eggs and ham.')
from spacy.symbols import nsubj, VERB
# Finding a verb with a subject from below — good
verbs = set()
for possible_subject in doc:
    if possible_subject.dep == nsubj and possible_subject.head.pos == VERB:
        verbs.add(possible_subject.head)
        print (possible_subject.dep_)       
print (verbs)

nsubj
{like}


In [23]:
## http://stackoverflow.com/questions/40288323/what-do-spacys-part-of-speech-and-dependency-tags-mean
## https://spacy.io/docs/usage/pos-tagging

In [24]:
# Let's look at the dependencies of this example:
example2 = "I love eggs and medication. Medication is good. I eat bread a lot"
parsedEx = parser(example2)
# shown as: original token, dependency tag, head word, left dependents, right dependents
for token in parsedEx:
    print(token.orth_, token.dep_, token.head.orth_, [t.orth_ for t in token.lefts], [t.orth_ for t in token.rights])

I nsubj love [] []
love ROOT love ['I'] ['eggs', '.']
eggs dobj love [] ['and', 'medication']
and cc eggs [] []
medication conj eggs [] []
. punct love [] []
Medication nsubj is [] []
is ROOT is ['Medication'] ['good', '.']
good acomp is [] []
. punct is [] []
I nsubj eat [] []
eat ROOT eat ['I'] ['bread', 'lot']
bread dobj eat [] []
a det lot [] []
lot npadvmod eat ['a'] []


In [26]:
example2 = "I love eggs and medication. Medication is good. I eat bread a lot"
parsedEx = parser(example2)
# shown as: original token, dependency tag, head word, left dependents, right dependents
for token in parsedEx:
    print(token.orth_, token.dep_, token.head.orth_)

I nsubj love
love ROOT love
eggs dobj love
and cc eggs
medication conj eggs
. punct love
Medication nsubj is
is ROOT is
good acomp is
. punct is
I nsubj eat
eat ROOT eat
bread dobj eat
a det lot
lot npadvmod eat
